# Run STARCOP models on raw EMIT data

>  V. Růžička, G. Mateo-Garcia, L. Gómez-Chova, A. Vaughan, L. Guanter, and A. Markham, [Semantic segmentation of methane plumes with hyperspectral machine learning models](https://www.nature.com/articles/s41598-023-44918-6). _Scientific Reports 13, 19999_ (2023). DOI: 10.1038/s41598-023-44918-6.

Demo with loading the AVIRIS trained models to show zero-shot generalisation on the data from EMIT.

In [1]:
!pip install git+https://github.com/spaceml-org/STARCOP.git
!pip install huggingface_hub[cli,torch]

  Cloning https://github.com/spaceml-org/STARCOP.git to /tmp/pip-req-build-bw7js4tl
  Running command git clone --filter=blob:none --quiet https://github.com/spaceml-org/STARCOP.git /tmp/pip-req-build-bw7js4tl
  Resolved https://github.com/spaceml-org/STARCOP.git to commit 549d294c2034ae9afdf223964412e66212313218
  Preparing metadata (setup.py) ... done


In [2]:
!pip install netcdf4

## Step 1: download EMIT image

In order to download and process the EMIT image we will use the emit reader in the [georeader](https://github.com/spaceml-org/georeader/) package. See [this tutorial](https://github.com/spaceml-org/georeader/blob/main/notebooks/emit_explore.ipynb) for an example of how to load and plot the data.

In [3]:
from huggingface_hub import hf_hub_download

In [4]:
from georeader.readers import emit

In [5]:
!pip install starcop

In [9]:
from starcop import models
print(dir(models))


['__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']


In [10]:
!pip install --upgrade starcop


In [11]:
!pip show starcop


Name: starcop
Version: 0.1.0
Summary: Semantic segmentation of methane plumes with hyperspectral machine learning models
Home-page: 
Author: Vit Rutzicka, Gonzalo Mateo-Garcia
Author-email: 
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: fsspec, gcsfs, geopandas, georeader-spaceml, hydra-core, ipykernel, ipython, kornia, matplotlib, omegaconf, pytorch-lightning, rasterio, scikit-image, scikit-learn, segmentation-models-pytorch, torch, torchmetrics, wandb
Required-by: 


In [12]:
import starcop
print(starcop.__version__)


0.1.0


In [13]:
import starcop
print(dir(starcop))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'models']


In [21]:
import starcop
print(dir(starcop.models))

['__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']


In [16]:
!pip install --upgrade starcop

In [22]:
from huggingface_hub import hf_hub_download
from georeader.readers import emit
from starcop.models import mag1c_emit
from georeader import plot
import starcop
from starcop.models.model_module import ModelModule
import netCDF4 as nc
import os
import torch
import omegaconf
import numpy as np
import matplotlib.pyplot as plt
from starcop.models.utils import padding
import georeader

link = emit.get_radiance_link("EMIT_L1B_RAD_001_20220827T060753_2223904_013.nc")

# Add the user and password of the NASA Earthdata portal (https://search.earthdata.nasa.gov/search)
product = emit.download_product(link,
                                auth=("user","password"))

rst = emit.EMITImage(product)
rst
from starcop.models.model_module import ModelModule
import netCDF4 as nc
import os
import torch
import omegaconf
import numpy as np
import matplotlib.pyplot as plt
from starcop.models.utils import padding
import georeader

link = emit.get_radiance_link("EMIT_L1B_RAD_001_20220827T060753_2223904_013.nc")

# Add the user and password of the NASA Earthdata portal (https://search.earthdata.nasa.gov/search)
product = emit.download_product(link,
                                auth=("user","password"))

rst = emit.EMITImage(product)
rst

ImportError: cannot import name 'mag1c_emit' from 'starcop.models' (unknown location)

In [18]:
import starcop
print(starcop.__file__)


/usr/local/lib/python3.11/dist-packages/starcop/__init__.py


## Step 2: Load RGB

The RGB bands used to train the model correspond to the wavelengths 640nm, 550nm and 460nm respectively.

In [19]:
pip install --upgrade starcop

In [20]:
wavelengths_read = np.array([640, 550, 460])

bands_read = np.argmin(np.abs(wavelengths_read[:, np.newaxis] - rst.wavelengths), axis=1).tolist()
rst_rgb = rst.read_from_bands(bands_read)
rst_rgb

NameError: name 'np' is not defined

In [ ]:
rgb_raw = rst_rgb.load_raw(transpose=True)
plt.imshow((rgb_raw/12).clip(0,1).transpose(1,2,0))

## Step 3: run mag1c on the EMIT product

Run mag1c filter retrieval based on the work of [Foote et al. 2020](https://ieeexplore.ieee.org/document/9034492).

In [ ]:
mfoutput, albedo = mag1c_emit.mag1c_emit(rst, column_step=2, georreferenced=False)
mfoutput

In [ ]:
plt.imshow(mfoutput, vmin=0,vmax=1750)
plt.title("$\Delta$CH$_4$ [ppm x m]")
plt.colorbar()

## Step 4: Load STARCOP model

In [ ]:
from huggingface_hub import hf_hub_download
# experiment_name = "hyperstarcop_mag1c_only"
experiment_name = "hyperstarcop_mag1c_rgb"
subfolder_local = f"models/{experiment_name}"
config_file = hf_hub_download(repo_id="isp-uv-es/starcop",subfolder=subfolder_local, filename="config.yaml",
                              local_dir=".", local_dir_use_symlinks=False)
model_file = hf_hub_download(repo_id="isp-uv-es/starcop",subfolder=subfolder_local,
                             filename="final_checkpoint_model.ckpt",
                              local_dir=".", local_dir_use_symlinks=False)

In [ ]:
# TODO upload to hugging face?
hsi_model_path = os.path.join(subfolder_local, "final_checkpoint_model.ckpt")
hsi_config_path =  os.path.join(subfolder_local, "config.yaml")

device = torch.device("cpu")
config_general = omegaconf.OmegaConf.load(os.path.join(os.path.dirname(os.path.abspath(starcop.__file__)), 'config.yaml'))


def load_model_with_emit(model_path, config_path):
    config_model = omegaconf.OmegaConf.load(config_path)
    config = omegaconf.OmegaConf.merge(config_general, config_model)

    model = ModelModule.load_from_checkpoint(model_path, settings=config)
    model.to(device)
    model.eval() # !

    print("Loaded model with",model.num_channels,"input channels")

    return model, config

hsi_model, hsi_config = load_model_with_emit(hsi_model_path, hsi_config_path)
print("successfully loaded HyperSTARCOP model!")

## Step 5: re-scale the input data

In [ ]:
# Data re-normalisation to fit the range of our models
# (these params were found from statistics of the training datasets and data ranges)

# DIV the EMIT data by
MAGIC_DIV_BY = 240.
RGB_DIV_BY = 20.
# clipping too large values
MAGIC_CLIP_TO = [0.,2.]
RGB_CLIP_TO =   [0.,2.]
# MULT_BY to get it back to the range we saw in the AVIRIS data ...
MAGIC_MULT_BY = 1750.
RGB_MULT_BY =   60.


# NORMALISE
# emit rgb has max ~22
e_mag1c = np.clip(mfoutput / MAGIC_DIV_BY, MAGIC_CLIP_TO[0], MAGIC_CLIP_TO[1]) * MAGIC_MULT_BY
e_rgb = np.clip(rgb_raw / RGB_DIV_BY, RGB_CLIP_TO[0], RGB_CLIP_TO[1]) * RGB_MULT_BY
input_data = np.concatenate([e_mag1c[None], e_rgb], axis=0)
input_data.shape

## Step 6: run inference

In [ ]:
pred = padding.padded_predict(input_data, model=lambda x: torch.sigmoid(hsi_model(x)))
pred.shape

In [ ]:
plt.imshow(pred[0],vmin=0,vmax=1)

## Step 7: georreference and plot results

In [ ]:
crs_utm = georeader.get_utm_epsg(rst.footprint("EPSG:4326"))
emit_image_utm = rst.to_crs(crs_utm)

In [ ]:
mfgeo = emit_image_utm.georreference(mfoutput, fill_value_default=-1)
predgeo = emit_image_utm.georreference(pred[0], fill_value_default=0)
rgbgeo = emit_image_utm.georreference(rgb_raw, fill_value_default=-1)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(18,6),sharey=True)

rgbgeomask = np.any(rgbgeo.values == -1, axis=0,keepdims=False)
rgbplot = (rgbgeo/12).clip(0,1)
rgbplot.values[:, rgbgeomask] = -1
plot.show(rgbplot, ax=ax[0], title= "RGB",mask=True, add_scalebar=True)
plot.show(mfgeo, ax=ax[1], title= "$\Delta$CH$_4$ [ppm x m]",mask=True,vmin=0, vmax=1750,
         add_colorbar_next_to=True, add_scalebar=True)
plot.show(predgeo, ax=ax[2], title= "pred", mask=True, vmin=0, vmax=1, add_scalebar=True,
          add_colorbar_next_to=True)

## Licence

The *STARCOP* database and all pre-trained models are released under a [Creative Commons non-commercial licence](https://creativecommons.org/licenses/by-nc/4.0/legalcode.txt). For using the models in comercial pipelines written consent by the authors must be provided.

 If you find this work useful please cite:
```
@article{ruzicka_starcop_2023,
	title = {Semantic segmentation of methane plumes with hyperspectral machine learning models},
	volume = {13},
	issn = {2045-2322},
	url = {https://www.nature.com/articles/s41598-023-44918-6},
	doi = {10.1038/s41598-023-44918-6},
	number = {1},
	journal = {Scientific Reports},
	author = {Růžička, Vít and Mateo-Garcia, Gonzalo and Gómez-Chova, Luis and Vaughan, Anna, and Guanter, Luis and Markham, Andrew},
	month = nov,
	year = {2023},
	pages = {19999}
}
```